# Напоминание

лучший друг: http://spark.apache.org/docs/latest/api/python/index.html

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import functions

In [2]:
sc = pyspark.SparkContext(appName='Domashka')
sql = pyspark.HiveContext(sc)

## Датасеты, которые понадобятся
### Alexa Millon
топ 1М сайтов в рейтинге

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

top_schema = StructType((
    StructField("rank", IntegerType(), nullable=False),
    StructField("site", StringType(), nullable=False),
))
alexa_top_df = sql.read.csv('/data/top-1m.csv', schema=top_schema)

In [4]:
alexa_top_df.show()

+----+---------------+
|rank|           site|
+----+---------------+
|   1|     google.com|
|   2|    youtube.com|
|   3|   facebook.com|
|   4|      baidu.com|
|   5|  wikipedia.org|
|   6|         qq.com|
|   7|     taobao.com|
|   8|      yahoo.com|
|   9|      tmall.com|
|  10|     amazon.com|
|  11|    twitter.com|
|  12|       sohu.com|
|  13|         jd.com|
|  14|       live.com|
|  15|    sina.com.cn|
|  16|  instagram.com|
|  17|      weibo.com|
|  18|         360.cn|
|  19|login.tmall.com|
|  20|     reddit.com|
+----+---------------+
only showing top 20 rows



### zgrab: результат сканирования интернета 

в этом датасете лежат результаты обхода интернета в поисках открытых портов 8888. Он как вы знаете используется блокнотиками Jupyter. А еще проксякми и веб-серверами

In [5]:
zgrab_content = sql.read.json('/data/zgrab.json').drop('_corrupt_record')

In [6]:
zgrab_content.show()

+--------------------+--------------------+---------------+--------------------+
|                data|               error|             ip|           timestamp|
+--------------------+--------------------+---------------+--------------------+
|               [[,]]|Get http://172.65...| 172.65.100.108|2018-05-13T02:30:...|
|[[, [<html style=...|                null| 192.230.66.113|2018-05-13T02:30:...|
|[[, [,,, [,,,,,,,...|                null|  159.65.233.69|2018-05-13T02:30:...|
|               [[,]]|Get http://168.25...| 168.255.128.19|2018-05-13T02:30:...|
|[[, [<!doctype ht...|                null| 185.162.171.15|2018-05-13T02:30:...|
|[[, [<html style=...|                null| 31.201.254.208|2018-05-13T02:30:...|
|[[, [<!DOCTYPE ht...|                null|  151.40.76.245|2018-05-13T02:30:...|
|               [[,]]|Get http://112.72...|  112.72.33.128|2018-05-13T02:30:...|
|[[, [<HTML><HEAD>...|                null| 218.145.166.91|2018-05-13T02:30:...|
|               [[,]]|Get ht

### resovled: IP-адреса сайтов из Alexa million

In [7]:
resolve_schema = (
    StructType()
    .add("host", "string")
    .add("ip", "string")
)
resolved = sql.read.csv("/data/ips", schema=resolve_schema)
resolved.show()

+------------+--------------+
|        host|            ip|
+------------+--------------+
|      qq.com| 111.161.64.40|
|      qq.com| 111.161.64.48|
|  google.com|216.58.212.238|
|facebook.com| 157.240.20.35|
| youtube.com|172.217.19.206|
|    sohu.com| 220.181.90.52|
|    sohu.com|123.125.116.28|
|    sohu.com|221.179.177.36|
|   yahoo.com|  98.137.246.7|
|   yahoo.com|  98.137.246.8|
|   yahoo.com|98.138.219.231|
|   yahoo.com|   72.30.35.10|
|   yahoo.com|98.138.219.232|
|   yahoo.com|    72.30.35.9|
| twitter.com|104.244.42.193|
| twitter.com| 104.244.42.65|
|     mail.ru|94.100.180.200|
|     mail.ru|217.69.139.202|
|     mail.ru|94.100.180.202|
|     mail.ru|217.69.139.199|
+------------+--------------+
only showing top 20 rows



# 1.1 Статистика по статусам ответов

напомним схему данных в zgrab:

In [8]:
zgrab_content.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- http: struct (nullable = true)
 |    |    |-- redirect_response_chain: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- body: string (nullable = true)
 |    |    |    |    |-- body_sha256: string (nullable = true)
 |    |    |    |    |-- content_length: long (nullable = true)
 |    |    |    |    |-- headers: struct (nullable = true)
 |    |    |    |    |    |-- accept_ranges: array (nullable = true)
 |    |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |    |-- access_control_allow_origin: array (nullable = true)
 |    |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |    |-- cache_control: array (nullable = true)
 |    |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |    |-- connection: array (nullable = true)
 |    |    |    |    |    |    |-- element: stri

вытащим из него данные по статусу ответов:

In [9]:
zgrad_codes = (
    zgrab_content
    .select("data.http.response.status_code")
)

zgrad_codes.show()

+-----------+
|status_code|
+-----------+
|       null|
|        503|
|        403|
|       null|
|        403|
|        200|
|        401|
|       null|
|        404|
|       null|
|        503|
|        200|
|        200|
|       null|
|       null|
|       null|
|        403|
|       null|
|       null|
|       null|
+-----------+
only showing top 20 rows



### Задание 1
посчитайте общую статистику: сколько было зафиксировано ответов с каждым статусом

Должно получиться что-то такое:

```
+-----------+-----+
|status_code|count|
+-----------+-----+
|       null|36390|
|        200| 7132|
|        404| 1971|
|        403| 1724|
|        503| 1242|
|        400|  658|
|        401|  576|
|        502|  142|
|        405|   35|
|        500|   32|
|        426|   29|
|        302|   23|
|        407|   11|
|        501|    5|
|        410|    2|
|        596|    1|
|        206|    1|
|        504|    1|
|        422|    1|
|        101|    1|
+-----------+-----+
```

## Задание 2

Отфильтруйте `zgrab_codes` так чтобы он не содержал пустых значений:

In [10]:
zgrab_codes_nonull = # ваш код здесь

zgrab_codes_nonull.show(5)

+-----------+
|status_code|
+-----------+
|        503|
|        403|
|        403|
|        200|
|        401|
+-----------+
only showing top 5 rows



## Задание 3
Посчитайте, сколько ответов сообщали об ошибке.

- `status_code` будет принимать значения от 400 до 499 при ошибке на стороне клиента (ошибка 4хх)

- `status_code` будет принимать значения от 500 до 599 при ошибке на стороне сервера (ошибка 5хх)

Посчитайте, сколько было было ошибок 4хх и 5хх

В ответе что-то такое:

```
+----------+-----+
|short_code|count|
+----------+-----+
|         5| 1423|
|         4| 5007|
+----------+-----+
```

## Задание 4

Определите IP-адреса для сайтов в рейтинге Алексы

В ответе такое:

```
+----+-------------+-------------+---------------+
|rank|         site|         host|             ip|
+----+-------------+-------------+---------------+
|   1|   google.com|   google.com| 216.58.212.238|
|   2|  youtube.com|  youtube.com| 172.217.19.206|
|   3| facebook.com| facebook.com|  157.240.20.35|
|   4|    baidu.com|    baidu.com|123.125.115.110|
|   4|    baidu.com|    baidu.com| 220.181.57.216|
|   5|wikipedia.org|wikipedia.org| 91.198.174.192|
|   6|       qq.com|       qq.com|  111.161.64.48|
|   6|       qq.com|       qq.com|  111.161.64.40|
|   7|   taobao.com|   taobao.com| 140.205.94.189|
|   7|   taobao.com|   taobao.com| 140.205.220.96|
|   8|    yahoo.com|    yahoo.com|     72.30.35.9|
|   8|    yahoo.com|    yahoo.com| 98.138.219.232|
|   8|    yahoo.com|    yahoo.com|    72.30.35.10|
|   8|    yahoo.com|    yahoo.com| 98.138.219.231|
|   8|    yahoo.com|    yahoo.com|   98.137.246.8|
|   8|    yahoo.com|    yahoo.com|   98.137.246.7|
|   9|    tmall.com|    tmall.com| 140.205.130.99|
|   9|    tmall.com|    tmall.com| 140.205.94.193|
|  10|   amazon.com|   amazon.com| 176.32.103.205|
|  10|   amazon.com|   amazon.com|205.251.242.103|
+----+-------------+-------------+---------------+
```

## Задание 5

Найдите, какие сайты из рейтинга алексы держат порт 8888 открытым (то есть присутствуют также в датасете `zgrab_content` и у них `status_code` – не `NULL`)


примерный ответ:

```
+-------------------+-----------+-----+---------------+
|               site|status_code| rank|             ip|
+-------------------+-----------+-----+---------------+
|   thedailystar.net|        503| 5976| 107.154.81.121|
|           comfy.ua|        503|13880|   45.60.72.216|
|nationalexpress.com|        503|17692| 149.126.74.191|
|   surveyjunkie.com|        503|29149|107.154.249.243|
|       tripcase.com|        503|33526|   45.60.46.204|
|           aota.org|        503|34832|   45.60.63.136|
|   translink.com.au|        503|35576|   45.60.15.189|
| princehotels.co.jp|        503|40954|   45.60.72.180|
|    lottery-win.org|        503|41344| 107.154.213.27|
|          cinesa.es|        503|57032|107.154.113.141|
+-------------------+-----------+-----+---------------+
```